In [1]:
# Specify variables
# sys.path.insert(local_haven)
from haven import haven_jupyter as hj
from haven import haven_results as hr
from haven import haven_utils as hu

# please define the path to the experiments
savedir_base = '/mnt/projects/vision_prototypes/synbols/logs_borgy_baselines/'
# savedir_base = '/mnt/projects/vision_prototypes/synbols/font_plain_borgy/'

exp_list = None

# exp_config_name = <exp_config_name>
# exp_list = hu.load_py(exp_config_name).EXP_GROUPS['mnist']

# get specific experiments, for example, {'model':'resnet34'}
filterby_list = None

# group the experiments based on a hyperparameter, for example, ['dataset']
groupby_list = None
verbose = 0

# plot vars
# y_metrics='train_loss'
x_metric='epoch'
# log_metric_list = ['train_loss']
map_exp_list = []
# title_list=['dataset']
# legend_list=['model']

# get experiments
rm = hr.ResultManager(exp_list=exp_list, 
                      savedir_base=savedir_base, 
                      filterby_list=filterby_list,
                      verbose=verbose
                     )
score_lists = rm.get_score_lists()
# score = rm.get_score_table(columns=['dataset', 'exp_id'])


# launch dashboard
hj.get_dashboard(rm, vars(), wide_display=True)

100%|██████████| 505/505 [00:00<00:00, 734.10it/s] 


<IPython.core.display.Javascript object>

Output()

In [2]:
import numpy as np

exp_list = rm.get_exp_list_df()

def get_hyperparameters(exp_list_df, min_threshold=2, max_threshold=None):
    exp_list_df.reset_index()
    if max_threshold is None:
        max_threshold = exp_list.shape[0] - 1
    column_count = []
    for column in exp_list.columns:
        _set = set([str(v) for v in exp_list[column].values])
        column_count.append(len(_set))
    indices = np.arange(len(exp_list.columns))
    column_count = np.array(column_count)
    indices = indices[(column_count >= min_threshold) & (column_count <= max_threshold)]
    hyperparameters = [exp_list.columns[i] for i in indices]
    return hyperparameters

In [3]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

def multipage(filename, figs=None, dpi=200):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()
multipage('synbols.pdf')

## Show accuracies

In [36]:
df = rm.get_score_df()
datasets = ["plain", "default_n=1000000_", "default_n=100000_", "mnist", "svhn", "camouflage", "tiny"]
models = ["mlp", "conv4", "resnet18", "resnet50", "vgg16", "warn"]
masks = ["'random'", "'stratified_font'", "'stratified_char'", "'compositional_char_font'"]
tasks = ["font", "char"]

def filter_dataset(dataframe, dset):
    if dataset in ["mnist", "svhn"]:
        df_dataset = dataframe[dataframe["dataset.name"].str.contains(dset)]
    else:
        df_dataset = dataframe[dataframe["dataset.path"].str.contains(dset)]
    return df_dataset
def filter_task(dataframe, task):
    if dataset not in ["mnist", "svhn"]:
        df_dataset = dataframe[dataframe["dataset.task"].str.contains(task)]
    else:
        df_dataset = dataframe
    return df_dataset 
def filter_mask(dataframe, mask):
    df_ood = dataframe[dataframe['dataset.mask'] == mask]
    return df_ood


for dataset in datasets:
    #     df_model = df[df["backbone.name"].str.contains(model)]
    df_dataset = filter_dataset(df, dataset)
    for task in tasks:
        df_task = filter_task(df_dataset, task)
        for mask in masks:
            if mask != "'random'" and task not in mask:
                continue
            print("Dset:%s, Task:%s, MASK:%s"% (dataset, task, mask))
            df_ood = filter_mask(df_task, mask)
#             df_ood = df_task
            for model in models:
                df_model = df_ood[df_ood["backbone.name"].str.contains(model)]
                df_model = df_model.groupby('seed')["val_accuracy (max)"].agg('max').reset_index()
                print(df_model["val_accuracy (max)"].mean(), df_model["val_accuracy (max)"].std())


Dset:plain, Task:font, MASK:'random'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:plain, Task:font, MASK:'stratified_font'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:plain, Task:font, MASK:'compositional_char_font'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:plain, Task:char, MASK:'random'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:plain, Task:char, MASK:'stratified_char'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:plain, Task:char, MASK:'compositional_char_font'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:default_n=1000000_, Task:font, MASK:'random'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:default_n=1000000_, Task:font, MASK:'stratified_font'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:default_n=1000000_, Task:font, MASK:'compositional_char_font'
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
Dset:default_n=1000000_, Task:char, MASK:'random'
0.5910016666666666 0.0008897518380612513
0.8077816666666666 0.000192

## Show times

In [17]:
df = rm.get_score_df()

datasets = ["plain", "default_n=1000000_", "default_n=100000_", "mnist", "svhn", "tiny"]
datasets = ["tiny"]
models = ["mlp", "conv4", "resnet18", "resnet50", "vgg16", "warn"]


for dataset in datasets:
#     df_model = df[df["backbone.name"].str.contains(model)]
    print(dataset)
    if dataset in ["mnist", "svhn"]:
        df_dataset = df[df["dataset.name"].str.contains(dataset)]
    else:
        df_dataset = df[df["dataset.path"].str.contains(dataset)]
    for model in models:
        df_model = df_dataset[df_dataset["backbone.name"].str.contains(model)]
        print(df_model["val_batch_time"].mean())

tiny
0.0018573602040608723
0.0030656655629475913
nan
nan
nan
nan
